# Sentence BERT(SBERT)

## 1. BERT의 문장 임베딩

BERT로부터 문장 벡터를 얻는 방법은 여러가지 방법이 존재하지만 여기서는 총 세 가지에 대해서 언급하겠습니다. 만약 사전 학습된 BERT에 'I love you'라는 문장이 입력된다고 하였을 때, 이 문장에 대한 벡터를 얻는 첫 번째 방법은 [CLS]토큰의 출력 벡터를 문장 벡터로 간주하는 것입니다.
<br />

![](https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC1.PNG)
<br />

앞서 BERT로 텍스트 분류 문제를 풀 때, [CLS]토큰의 출력 벡터를 출력층의 입력으로 사용했던 점을 상기해봅시다. 이는 [CLS]토큰이 입력된 문장에 대한 총체적 표현이라고 간주하고 있기 때문입니다. 다시 말해 [CLS]토큰 자체를 입력 문장의 벡터로 간주할 수 있습니다.
<br />

![](https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC2.PNG)
<br />

문장 벡터를 얻는 두 번째 방법은 [CLS] 토큰만을 사용하는 것이 아니라 BERT의 모든 출력 벡터들을 평균내는 것입니다. 위 그림에서는 출력 벡터들의 평균을 'pooling'이라고 표현했습니다. 이를 평균 풀링(mean pooling)을 하였다고 표현하기도 합니다. 그런데 풀링에는 평균 풀링만 있는 것이 아니라 맥스 풀링(max pooling)또한 존재합니다. 정리하자면 사전 학습된 BERT로부터 문장 벡터를 얻는 방법은 다음과 같이 세 가지가 있습니다.
<br />

- BERT의 [CLS] 토큰의 출력 벡터를 문장 벡터로 간주한다.

- BERT의 모든 단어의 출력 벡터에 대해서 평균 풀링을 수행한 벡터를 문장 벡터로 간주한다.

- BERT의 모든 단어의 출력 벡터에 대해서 맥스 풀링을 수행한 벡터를 문장 벡터로 간주한다.
<br />


## 2. SBERT(Sentence BERT)

### 1) 문장 쌍 분류 태스크로 파인 튜닝

SBERT를 학습하는 첫 번째 방법은 문장 쌍 분류 태스크. 대표적으로는 NLI(Natural Language Inferencing)문제를 푸는 것입니다. NLI는 두 개의 문장이 주어지면 entailment, contradiction, neutral인지 맞추는 문제입니다.
<br />

|문장 A|문장 B|레이블|
|:---:|:---:|:---:|
|A lady sits on a bench that is against a shopping mall.| A person sits on the seat.| Entailment|
|A lady sits on a bench that is against a shopping mall.| A woman is sitting against a building.| Entailment|
|A lady sits on a bench that is against a shopping mall.| Nobody is sitting on the bench.| Contradiction|
|Two women are embracing while holding to go packages.| The sisters are hugging goodbye while holding to go packages after just eating lunch| Neutral|
<br />

SBERT는 NLI 데이터를 학습하기 위해 다음과 같은 구조를 가집니다.
<br />

![](https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC3.PNG)
<br />

우선 문장 A와 문장 B 각각을 BERT의 입력으로 넣고, 평균 풀링 또는 맥스 풀링을 통해서 각각에 대한 문장 임베딩 벡터를 얻습니다. 여기서 이를 각각 u, v라고 하겠습니다. 그리고 나서 u와 v의 차이 벡터를 구합니다.(|u - v|) 다음으로 이 세 벡터를 연결(concatenation)합니다. 해당 수식은 아래와 같습니다.
<br />

$$h = (u;\; v;\; |u-v|)$$
<br />

만약 BERT의 문장 임베딩 벡터의 차원이 n이라면 세 개의 벡터를 연결한 벡터 h의 차원은 3n이 됩니다.
<br />

![](https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC4.PNG)
<br />

그리고 이 벡터를 출력층으로 보내 Multi-Class Classification문제를 풀도록 합니다. 분류하고자 하는 클래스의 개수가 k라면, 가중치 행렬 $3n \times k$의 크기를 가지는 행렬 $W_y$를 곱한 후에 소프트맥스 함수를 통과시킨다고도 볼 수 있습니다. 식은 아래와 같습니다.
<br />

$$o = \text{softmax}(W_yh)$$

### 2) 문장 쌍 회귀 태스크로 파인 튜닝

SBERT를 학습하는 두 번째 방법은 문장 쌍으로 회귀 문제를 푸는 것으로 대표적으로 STS(Semantic Textual Similarity)문제를 푸는 경우입니다. STS란 두 개의 문장으로부터 의미적 유사성을 구하는 문제를 말합니다. 아래는 그 예시이고 레이블은 두 문장의 유사도로 범위는 0~5입니다.
<br />


|문장 A|문장 B|레이블|
|:---:|:---:|:---:|
|A plane is taking off|An air plane is taking off|5.00|
|A man is playing a large flute.|A man is playing a flute|3.80|
|A man is spreading shreded cheese on a pizza.|A man is spreading shredded cheese on an uncoo...|3.80|
|Three man are playing chess.|Two men are playing chess|2.60|
|A man is playing the cello.|A man seated is playing the cello.|4.25|

<br />

SBERT는 STS 데이터를 학습하기 위해 다음과 같은 구조를 가집니다.
<br />

![](https://wikidocs.net/images/page/156176/%EA%B7%B8%EB%A6%BC5.PNG)
<br />

문장 A와 문장 B를 각각 BERT의 입력으로 넣고, 평균 불링 또는 맥스 풀링을 통해서 각각에 대한 문장 임베딩 벡터를 얻습니다. 이를 각각 u와 v라고 하였을 때 이 두 벡터의 코사인 유사도를 구합니다. 그리고 해당 유사도와 레이블 유사도와의 평균 제곱 오차(Mean Squared Error, MSE)를 최소화하는 방식으로 학습합니다. 